In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy import stats

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

from transformers import BertTokenizer, BertModel
from tqdm import tqdm

import torch
import torch.nn as nn

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)

In [2]:
train_df = pd.read_csv('clean_data.csv')
val_df = pd.read_csv('clean_val.csv')
test_df = pd.read_csv('clean_test.csv')

In [3]:
train_df.dropna(inplace = True)

In [4]:
with open("numerical.txt", "r", encoding="utf-8") as file:
    numeric_features = [line.strip() for line in file.readlines()]
    
with open("cat.txt", "r", encoding="utf-8") as file:
    cat_features = [line.strip() for line in file.readlines()]
    
with open("text.txt", "r", encoding="utf-8") as file:
    text_features = [line.strip() for line in file.readlines()]

In [5]:
print('Total number of Numerical Features and one target:', len(numeric_features))
print('Total number of Categorical Features:',len(cat_features))
print('Total number of Text Features:',len(text_features))

Total number of Numerical Features and one target: 9
Total number of Categorical Features: 70
Total number of Text Features: 2


In [6]:
def combine_text(df):
    df['combined_text'] = df['summary'] + " [SEP] "  + df['host_about'] + " [SEP] "
    return 

In [7]:
combine_text(train_df)
combine_text(val_df)
combine_text(test_df)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [9]:
class BoxCoxTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.lambdas_ = {}

    def fit(self, X, y=None):
        X = X.copy()
        for column in X.columns:
            if any(X[column] <= 0):
                X[column] = X[column] + abs(X[column].min()) + 1
            _, lambda_optimal = stats.boxcox(X[column])
            self.lambdas_[column] = lambda_optimal
        return self

    def transform(self, X):
        X = X.copy()
        for column, lambda_optimal in self.lambdas_.items():
            if any(X[column] <= 0):
                X[column] = X[column] + abs(X[column].min()) + 1
            X[column] = stats.boxcox(X[column], lmbda=lambda_optimal)
        return X

def get_bert_embeddings(text_series, batch_size):
    embeddings = []
    model.eval()  
    
    for i in range(0, len(text_series), batch_size):
        batch_texts = text_series[i:i + batch_size]
        
        if not batch_texts: 
            continue
        
        inputs = tokenizer(batch_texts, return_tensors='pt', truncation=True, padding=True, max_length=128)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(batch_embeddings)
    
    if embeddings:
        return np.vstack(embeddings)
    else:
        return np.array([])
    
def concat(df, text_features, batch_size, pooling, is_train = False):

    embeddings_list = []

    for feature in text_features:
        print(feature)
        text_embeddings = get_bert_embeddings(df[feature].tolist(), batch_size)

        if pooling == 'max':
            text_embeddings = np.max(text_embeddings, axis=1)  
        elif pooling == 'mean':
            text_embeddings = np.mean(text_embeddings, axis=1)  

        if text_embeddings.ndim == 1:
            text_embeddings = text_embeddings.reshape(-1, 1)

        embeddings_list.append(text_embeddings)
        
    if is_train:
        transformed_features = preprocessor.fit_transform(df).toarray()
    else:
        transformed_features = preprocessor.transform(df).toarray()

    combined_features = np.concatenate(embeddings_list + [transformed_features], axis=1)

    return combined_features

In [10]:
numeric_transformer = Pipeline([
    ('boxcox', BoxCoxTransformer()),
    ('scaler', StandardScaler())
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

In [11]:
text_features = ['combined_text']

In [12]:
train_combined_features = concat(train_df, text_features, batch_size = 32, pooling = 'max', is_train = True)
val_combined_features = concat(val_df, text_features, batch_size = 32, pooling = 'max', is_train = False)
test_combined_features = concat(test_df, text_features, batch_size = 32, pooling = 'max', is_train = False)

combined_text
combined_text
combined_text


In [13]:
train_y = train_df['price']
val_y = val_df['price']
test_y = test_df['price']

In [14]:
from tqdm import tqdm

def rmse(y, y_predict):
    mse = mean_squared_error(y, y_predict)
    return math.sqrt(mse)

def evaluate_models(model):
    results = []

    for name, model in tqdm(model, desc="Evaluating Models"):
        
        model.fit(train_combined_features, train_y)
        
        val_predict = model.predict(val_combined_features)
        
        val_mae = mean_absolute_error(val_y, val_predict)
        val_rmse = rmse(val_y, val_predict)
        
        
        test_predict = model.predict(test_combined_features)
        test_mae = mean_absolute_error(test_y, test_predict)
        test_rmse = rmse(test_y, test_predict)
       
        results.append({
            'model': name,
            'val_mae': val_mae,
            'val_rmse': val_rmse,
            'test_mae': test_mae,
            'test_rmse': test_rmse,
        })
    
    return results

In [15]:
xgb = XGBRegressor()
lgm = LGBMRegressor()
cat = CatBoostRegressor()

models = [('xgb', xgb), ('lgm', lgm),('cat', cat)]

In [16]:
res = evaluate_models(models)

Evaluating Models:  67%|███████████████████████████████████████████▎                     | 2/3 [00:04<00:01,  1.83s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002956 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1537
[LightGBM] [Info] Number of data points in the train set: 17941, number of used features: 303
[LightGBM] [Info] Start training from score 134.868792
Learning rate set to 0.064608
0:	learn: 99.0759626	total: 149ms	remaining: 2m 28s
1:	learn: 96.4701277	total: 157ms	remaining: 1m 18s
2:	learn: 94.3310097	total: 164ms	remaining: 54.6s
3:	learn: 92.2117454	total: 172ms	remaining: 42.8s
4:	learn: 90.2298667	total: 179ms	remaining: 35.7s
5:	learn: 88.4724399	total: 186ms	remaining: 30.9s
6:	learn: 86.9136734	total: 194ms	remaining: 27.5s
7:	learn: 85.5287161	total: 202ms	remaining: 25s
8:	learn: 84.1889704	total: 210ms	remaining: 23.1s
9:	learn: 82.9918290	total: 218ms	remaining: 21.6s
10:	learn: 81.8951536	total: 226ms	remaining: 20.3s

Evaluating Models: 100%|█████████████████████████████████████████████████████████████████| 3/3 [00:17<00:00,  5.76s/it]


In [17]:
res_df = pd.DataFrame(res)
res_df.sort_values(by = 'test_mae')

,model,val_mae,val_rmse,test_mae,test_rmse
2,cat,49.012240,274.980057,45.233306,134.920939
1,lgm,49.870846,274.508231,45.909430,137.820441
0,xgb,50.100984,275.244247,46.141323,134.417985


In [18]:
import optuna

def objective(trial):
    
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.03)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    n_estimators = trial.suggest_int('n_estimators', 1000, 2000)
    subsample = trial.suggest_float('subsample', 0.6, 1.0)
    alpha =  trial.suggest_loguniform('alpha', 1e-8, 10.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.4, 1.0)
   
    model = XGBRegressor(
                learning_rate=learning_rate,
                max_depth=max_depth,
                n_estimators=n_estimators,
                subsample=subsample,
                alpha = alpha,
                colsample_bytree = colsample_bytree,
                random_state=42
            )
    
    model.fit(train_combined_features, train_y)
  
    val_predict = model.predict(val_combined_features)

    val_mae = mean_absolute_error(val_y, val_predict)
    val_rmse = rmse(val_y, val_predict)
    
    return val_mae

In [19]:
study3 = optuna.create_study(direction='minimize')
study3.optimize(objective, n_trials=50) 

[I 2024-11-18 11:23:37,397] A new study created in memory with name: no-name-c39854d7-78c0-47f7-ad1c-fa27ca6439e9
[I 2024-11-18 11:24:07,710] Trial 0 finished with value: 50.69014963667936 and parameters: {'learning_rate': 0.020015192112312393, 'max_depth': 3, 'n_estimators': 1828, 'subsample': 0.8339553113903999, 'alpha': 3.8832754293605087, 'colsample_bytree': 0.5905197663271626}. Best is trial 0 with value: 50.69014963667936.
[I 2024-11-18 11:24:49,779] Trial 1 finished with value: 47.9058907937941 and parameters: {'learning_rate': 0.027987575477312003, 'max_depth': 10, 'n_estimators': 1183, 'subsample': 0.7964573293969506, 'alpha': 9.441915969257341e-07, 'colsample_bytree': 0.4393084422899785}. Best is trial 1 with value: 47.9058907937941.
[I 2024-11-18 11:26:02,248] Trial 2 finished with value: 47.77406798797661 and parameters: {'learning_rate': 0.01976853636531594, 'max_depth': 8, 'n_estimators': 1606, 'subsample': 0.6740542676849484, 'alpha': 0.044836123358310144, 'colsample_byt

In [20]:
xgb_best_param = study3.best_trial.params
print("Best trial for XGBRegressor:")
print(study3.best_trial.params)
print("Best Validation MAE:", study3.best_value)

Best trial for XGBRegressor:
{'learning_rate': 0.016640138218033517, 'max_depth': 9, 'n_estimators': 1357, 'subsample': 0.8093119211362332, 'alpha': 0.001498743353582768, 'colsample_bytree': 0.6488365154625855}
Best Validation MAE: 47.32220646967052


In [21]:
best_xgb = XGBRegressor(**xgb_best_param)
best_models = [('xgb', best_xgb)]

In [22]:
res = evaluate_models(best_models)

Evaluating Models: 100%|█████████████████████████████████████████████████████████████████| 1/1 [01:00<00:00, 60.19s/it]


In [23]:
res_df = pd.DataFrame(res)
res_df.sort_values(by = 'test_mae')

,model,val_mae,val_rmse,test_mae,test_rmse
0,xgb,47.606311,274.235724,44.208831,137.736277
